In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as sf

import numpy as np
if __name__ == "__main__":
    sc = SparkContext()


#Extract Bike
    lines = sc.textFile('citibike.csv', use_unicode=False).cache()

    def extractBike(partId, parts):
        if partId == 0:
            next(parts)
        import csv
        reader = csv.reader(parts)
    
    
        for row in reader:
        
            if row[6] == 'Greenwich Ave & 8 Ave':
                temp = row[3][:-3].split(" ")[1].split(':')
                if row[3].split(" ")[0].split('-')[2] == '01':
                    (bid, starttime, s, lat, lon) = (row[0], row[3][:-3], float(temp[0])*3600 + float(temp[1])*60 + float(temp[2]), row[7], row[8])
                    yield (bid, starttime, s,lat, lon)

    bike = lines.mapPartitionsWithIndex(extractBike)

#Extract Taxi

#GAlatlon = (40.73901691, -74.0026376)

    lines = sc.textFile("yellow.csv.gz", use_unicode=False).cache()
    GAlatlon = (40.73901691, -74.0026376)


    import math
    def distance(origin, destination):
        lat1, lon1 = origin
        lat2, lon2 = destination
        radius = 3959 # km

        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c

        return d




    def extractTaxi(partId, records):
        if partId == 0:
            records.next()
        import csv
        reader = csv.reader(records)
        for row in reader:
            if row[4] == "NULL" or row[5] == "NULL":
                continue
            origin = (float(row[4]), float(row[5]))
            if distance(origin, GAlatlon) <= 0.25:
                yield (row[1][:-2],row[2])
            


    taxi= lines.mapPartitionsWithIndex(extractTaxi)

#taxi.collect()

#compare time
    import datetime
    for i in taxi.collect():
        a = datetime.datetime.strptime(i[0], '%Y-%m-%d %H:%M:%S')
    
    for i in bike.collect():
        b = datetime.datetime.strptime(i[1], '%Y-%m-%d %H:%M:%S')
    


    taxi = taxi.map(lambda x : datetime.datetime.strptime(x[0], '%Y-%m-%d %H:%M:%S')).collect()

    bike = bike.map(lambda x : datetime.datetime.strptime(x[1], '%Y-%m-%d %H:%M:%S')).collect()

    s=0

    for i in taxi:
        for t in bike:
            if t > i:
                temp = t - i
                if temp.seconds < 600:
                    s += 1
                
    print s
    
sc.stop()               

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /Users/anaconda/lib/python2.7/site-packages/IPython/utils/py3compat.py:289 

In [1]:
sc

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import requests


In [6]:
tdata = sqlContext(taxi)

NameError: name 'taxi' is not defined